In [1]:
from openai import OpenAI
from dotenv import load_dotenv

from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

load_dotenv()
client = OpenAI()

In [6]:
import PyPDF2
import pandas as pd
import json
import time
import re

load_dotenv()
client = OpenAI()

# --- Excel Utilities ---
def parse_r1d_excel(file_path):
    # Load raw Excel (no headers assumed)
    df_raw = pd.read_excel(file_path, header=None, dtype=str)
    
    # Define the key headers we must find
    required_headers = ["Line#", "Type", "PE#", "Project#", 
                        "PE/Project/Accomplishments/Planned Programs Title", "BA#"]
    
    header_row = None
    
    # Search first 30 rows for header row
    for i in range(min(30, len(df_raw))):
        row_values = df_raw.iloc[i].fillna("").astype(str).str.strip().tolist()
        if all(any(req in cell for cell in row_values) for req in required_headers):
            header_row = i
            break
    
    if header_row is None:
        raise ValueError("❌ Could not find header row with required columns in first 30 rows.")
    
    # Reload Excel with proper header row
    df = pd.read_excel(file_path, header=header_row, dtype=str)
    
    # Clean column names (strip whitespace)
    df.columns = [str(c).strip() for c in df.columns]
    
    # Convert entire DataFrame to list of dicts
    data = df.fillna("").to_dict(orient="records")
    
    return data

# --- PDF Utilities ---
def read_pdf(file_path, max_pages=None):
    """Extract text from a PDF up to max_pages."""
    text = ""
    with open(file_path, "rb") as f:
        reader = PyPDF2.PdfReader(f)
        max_pages = max_pages if max_pages else len(reader.pages)
        for i in range(min(len(reader.pages), max_pages)):
            page = reader.pages[i]
            text += f"[[PAGE {i}]]" + (page.extract_text() or "") + "\n"
    return text

def read_pdf_range(file_path, start_page=None, end_page=None):
    """Extract text from a PDF within a page range."""
    text = ""
    with open(file_path, "rb") as f:
        reader = PyPDF2.PdfReader(f)
        num_pages = len(reader.pages)
        start = max(0, start_page if start_page is not None else 0)
        end = min(num_pages - 1, end_page if end_page is not None else num_pages - 1)
        for i in range(start, end + 1):
            page = reader.pages[i]
            text += f"[[PAGE {i}]]\n" + (page.extract_text() or "") + "\n"
    return text

# --- JSON Utilities ---
def get_json(string):
    """Parse a string or dict to JSON, handling code block formatting."""
    try:
        if isinstance(string, dict):
            return string
        return json.loads(string)
    except (json.JSONDecodeError, TypeError):
        try:
            cleaned = re.sub(r"^```(?:json)?\s*|\s*```$", "", string.strip(), flags=re.DOTALL)
            return json.loads(cleaned)
        except Exception as e:
            raise json.JSONDecodeError(f"Failed to parse JSON: {string[:100]}...", string, 0) from e

# --- OpenAI API Helper ---
def responses_call(user_query, system_message="You are a helpful assistant."):
    """Send a prompt to OpenAI Responses API and return the output text."""
    response = client.responses.create(
        model="gpt-4.1-mini",
        input=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": user_query}
        ]
    )
    return response.output[0].content[0].text

# --- LangChain Prompt Templates ---
query_pages_template = """
From the raw extract, find the page number for each PE from the PE. 

raw extract:
{pdf_text}

PE table:
{PE_table}

Make the output a list of JSONs with these keys:
PE#, PE/Project/Accomplishments/Planned Programs Title, start_page, end_page
"""

query_descriptions_template = """
From the raw extract, find the Description and full text (everything in that section excluding Description and Title) associated with each A/PP item in the A/PP table. 

raw extract:
{pdf_text}

A/PP table:
{PE_table}

Make the output a list of JSONs with these keys:
Line#, Type, PE#, Project#, PE/Project/Accomplishments/Planned Programs Title, BA#, Description, Full_Text
"""

llm = ChatOpenAI(model="gpt-4.1-mini")
query_pages_prompt = PromptTemplate(
    input_variables=["pdf_text", "PE_table"],
    template=query_pages_template
)
query_descriptions_prompt = PromptTemplate(
    input_variables=["pdf_text", "PE_table"],
    template=query_descriptions_template
)

# --- Data Extraction Functions ---
def extract_excel_data(excel_path):
    """Parse Excel and return DataFrame and JSON."""
    excel_content = parse_r1d_excel(excel_path)
    with open("indexed_all_columns.json", "w") as f:
        json.dump(excel_content, f, indent=2)
    print("✅ Extracted ALL columns into indexed_all_columns.json")
    return excel_content

def extract_pdf_text(pdf_path, max_pages=None):
    """Extract and save PDF text."""
    pdf_text = read_pdf(pdf_path, max_pages)
    with open("pdf_text_output.txt", "w", encoding="utf-8") as f:
        f.write(pdf_text)
    print("✅ Extracted PDF text into pdf_text_output.txt")
    return pdf_text

def get_pe_pages(pdf_text, pe_table):
    """Run LLMChain to get PE page numbers."""
    chain = LLMChain(llm=llm, prompt=query_pages_prompt)
    response = chain.run(pdf_text=pdf_text, PE_table=pe_table)
    return json.loads(response)

def query_descriptions(pe_row, pdf_path, df):
    """Query descriptions for a PE row using LLMChain."""
    start = time.time()
    pe = pe_row['PE#']
    start_page = pe_row['start_page']
    end_page = pe_row['end_page']
    PE_table = df[df["PE#"] == pe]
    pdf_text = read_pdf_range(pdf_path, start_page=start_page-10, end_page=end_page+10)
    chain = LLMChain(llm=llm, prompt=query_descriptions_prompt)
    response = chain.run(pdf_text=pdf_text, PE_table=PE_table)
    print(f"⏱️ Query time: {time.time() - start:.2f}s")
    return response

def clean_descriptions_dict(data):
    """Parse batch responses and separate failed ones."""
    master_list, failed_list = [], []
    for key, value in data.items():
        try:
            master_list.extend(get_json(value))
        except Exception:
            failed_list.append(key)
    return master_list, failed_list

# --- Main Processing Loop ---
def batch_query_descriptions(pages_json, df, pdf_path, max_loops=3, batch_size=3):
    """Batch query descriptions, retrying failed ones."""
    descriptions = []
    to_parse = pages_json[:batch_size]
    for loop in range(max_loops):
        if not to_parse:
            break
        print(f"\n🔄 Loop {loop+1} | Attempting {len(to_parse)} rows")
        responses_batch = {}
        for pe_row in to_parse:
            pe = pe_row["PE#"]
            try:
                responses_batch[pe] = query_descriptions(pe_row, pdf_path, df)
                print(f"✅ Parsed {pe}")
            except Exception as e:
                print(f"❌ Error for {pe}: {e}")
            time.sleep(10)
        # Clean results
        parsed, pe_unparsed = clean_descriptions_dict(responses_batch)
        descriptions.extend(parsed)
        if pe_unparsed:
            print(f"⚠️  Failed to parse (loop {loop+1}): {pe_unparsed}")
        else:
            print("🎉 All parsed successfully this round!")
        # Prepare next round
        to_parse = [row for row in pages_json if row["PE#"] in pe_unparsed]
    print(f"\n✨ Done. Parsed: {len(descriptions)}, Still unparsed: {to_parse and [r['PE#'] for r in to_parse]}")
    return descriptions

# --- DataFrame Update ---
def update_excel_with_descriptions(excel_df, app_descriptions_df):
    """Merge descriptions into Excel DataFrame."""
    updated_df = excel_df.merge(
        app_descriptions_df[["PE#", "Project#", "PE/Project/Accomplishments/Planned Programs Title", "Description", "Full_Text"]],
        on=["PE#", "Project#", "PE/Project/Accomplishments/Planned Programs Title"],
        how="left",
        suffixes=("", "_new")
    )
    mask = (updated_df["Type"] == "A/PP") & (updated_df["Description"].isin(["", None]))
    updated_df.loc[mask, "Description"] = updated_df.loc[mask, "Description_new"]
    updated_df = updated_df.drop(columns=["Description_new"])
    return updated_df


In [7]:
# --- Example Usage ---
excel_path = "FY25 Air Force RDTE Vol1 Exhibit_R-1D.xlsx"
pdf_path = "FY25 Air Force Research, Development, Test and Evaluation Vol I.pdf"

excel_df = pd.DataFrame(extract_excel_data(excel_path))
pdf_text = extract_pdf_text(pdf_path, max_pages=None)

pe_table = excel_df[excel_df['Type'] == "PE"]["PE/Project/Accomplishments/Planned Programs Title"]
pages_json = get_pe_pages(pdf_text, pe_table)
pages_df = pd.DataFrame(pages_json)

✅ Extracted ALL columns into indexed_all_columns.json
✅ Extracted PDF text into pdf_text_output.txt


/var/folders/nn/j70pmkwn6lx7kmclphdnwc_w0000gn/T/ipykernel_83757/3315774868.py:149: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=query_pages_prompt)
/var/folders/nn/j70pmkwn6lx7kmclphdnwc_w0000gn/T/ipykernel_83757/3315774868.py:150: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = chain.run(pdf_text=pdf_text, PE_table=pe_table)


In [20]:
display(excel_df.shape)
excel_df.head(40)

(3004, 17)

,Line#,Type,PE#,Project#,PE/Project/Accomplishments/Planned Programs Title,BA#,Description,Prior Years,FY 2023,FY 2024,FY 2025 Base,FY 2025 OCO,FY 2025 Total,FY 2026,FY 2027,FY 2028,FY 2029
0,1,PE,0601102F,,Defense Research Sciences,01,Defense Research Sciences consists of basic re...,0,377.616,401.486,361.93,0,361.93,308.87,333.612,382.456,405.8
1,1,Project,0601102F,613001,Physics and Electronics,1,Basic research in the Physics and Electronics ...,0,110.951,117.74,102.282,0,102.282,89.962,95.019,109.404,115.176
2,1,A/PP,0601102F,613001,Complex Electronics and Fundamental Quantum Pr...,1,,,41.262,47.096,49.454,0,49.454,,,,
3,1,A/PP,0601102F,613001,"Lasers and Optics, Electromagnetics, Communica...",1,,,40.231,45.919,37.08,0,37.08,,,,
4,1,A/PP,0601102F,613001,Plasma Physics and High Energy Density Non-Equ...,1,,,21.663,24.725,15.748,0,15.748,,,,
5,Accomplishments/Planned Programs (A/PP) Totals...,,,,,,,,103.15599999999999,117.73999999999998,102.282,0,102.282,,,,
6,1,CA,0601102F,613001,Program Increase - basic research,1,,,7.795,0,,,,,,,
7,Congressional Add Totals (sum of all CA's for ...,,,,,,,,7.795,0,,,,,,,
8,1,Project,0601102F,613002,"Aerospace, Chemical and Material Sciences",1,"Basic research in the Aerospace, Chemical, and...",0,112.79,117.926,107.377,0,107.377,95.819,104.896,118.342,126.123
9,1,A/PP,0601102F,613002,Aero-Structure Interactions and Control,1,,,31.499,35.378,36.236,0,36.236,,,,


In [25]:
display(pe_table.shape)
pages_df

(272,)

,PE#,PE/Project/Accomplishments/Planned Programs Title,start_page,end_page
0,0601102F,Defense Research Sciences,68,89
1,0601103F,University Research Initiatives,90,94
2,0602020F,Future AF Capabilities Applied Research,95,101
3,0602022F,University Affiliated Research Center (UARC) -...,102,104
4,0602102F,Materials,107,125
5,0602201F,Aerospace Vehicle Technologies,128,143
6,0602202F,Human Effectiveness Applied Research,146,170
7,0602203F,Aerospace Propulsion,172,206
8,0602204F,Aerospace Sensors,208,234
9,0602212F,"Defense Laboratories R&D Projects (10 U.S.C, S...",235,236


In [9]:
descriptions = batch_query_descriptions(pages_json, excel_df, pdf_path, batch_size=3)
app_descriptions_df = pd.DataFrame(descriptions)
app_descriptions_df.head()


🔄 Loop 1 | Attempting 3 rows
⏱️ Query time: 79.92s
✅ Parsed 0601102F
⏱️ Query time: 20.77s
✅ Parsed 0601103F
⏱️ Query time: 38.37s
✅ Parsed 0602020F
🎉 All parsed successfully this round!

✨ Done. Parsed: 21, Still unparsed: []


,Line#,Type,PE#,Project#,PE/Project/Accomplishments/Planned Programs Title,BA#,Description,Full_Text
0,2,A/PP,0601102F,613001,Complex Electronics and Fundamental Quantum Pr...,1,Scientific focus areas are atomic and molecula...,FY 2024 Plans:\nExplore a wide range of comple...
1,3,A/PP,0601102F,613001,"Lasers and Optics, Electromagnetics, Communica...",1,Scientific focus areas are physical mathematic...,FY 2024 Plans:\nExplore all aspects of produci...
2,4,A/PP,0601102F,613001,Plasma Physics and High Energy Density Non-Equ...,1,Scientific focus areas are plasma and electro-...,FY 2024 Plans:\nExplore a wide range of activi...
3,9,A/PP,0601102F,613002,Aero-Structure Interactions and Control,1,Scientific focus areas are high temperature ae...,FY 2024 Plans:\nInvestigate the characterizati...
4,10,A/PP,0601102F,613002,Complex Materials and Structures,1,"Scientific focus areas are design, manufacturi...",FY 2024 Plans:\nInvestigate multifunctional ma...


In [26]:
updated_excel_df = update_excel_with_descriptions(excel_df, app_descriptions_df)
updated_excel_df.head(20)

,Line#,Type,PE#,Project#,PE/Project/Accomplishments/Planned Programs Title,BA#,Description,Prior Years,FY 2023,FY 2024,FY 2025 Base,FY 2025 OCO,FY 2025 Total,FY 2026,FY 2027,FY 2028,FY 2029,Full_Text
0,1,PE,0601102F,,Defense Research Sciences,01,Defense Research Sciences consists of basic re...,0,377.616,401.486,361.93,0,361.93,308.87,333.612,382.456,405.8,NaN
1,1,Project,0601102F,613001,Physics and Electronics,1,Basic research in the Physics and Electronics ...,0,110.951,117.74,102.282,0,102.282,89.962,95.019,109.404,115.176,NaN
2,1,A/PP,0601102F,613001,Complex Electronics and Fundamental Quantum Pr...,1,Scientific focus areas are atomic and molecula...,,41.262,47.096,49.454,0,49.454,,,,,FY 2024 Plans:\nExplore a wide range of comple...
3,1,A/PP,0601102F,613001,"Lasers and Optics, Electromagnetics, Communica...",1,Scientific focus areas are physical mathematic...,,40.231,45.919,37.08,0,37.08,,,,,FY 2024 Plans:\nExplore all aspects of produci...
4,1,A/PP,0601102F,613001,Plasma Physics and High Energy Density Non-Equ...,1,Scientific focus areas are plasma and electro-...,,21.663,24.725,15.748,0,15.748,,,,,FY 2024 Plans:\nExplore a wide range of activi...
5,Accomplishments/Planned Programs (A/PP) Totals...,,,,,,,,103.15599999999999,117.73999999999998,102.282,0,102.282,,,,,NaN
6,1,CA,0601102F,613001,Program Increase - basic research,1,,,7.795,0,,,,,,,,NaN
7,Congressional Add Totals (sum of all CA's for ...,,,,,,,,7.795,0,,,,,,,,NaN
8,1,Project,0601102F,613002,"Aerospace, Chemical and Material Sciences",1,"Basic research in the Aerospace, Chemical, and...",0,112.79,117.926,107.377,0,107.377,95.819,104.896,118.342,126.123,NaN
9,1,A/PP,0601102F,613002,Aero-Structure Interactions and Control,1,Scientific focus areas are high temperature ae...,,31.499,35.378,36.236,0,36.236,,,,,FY 2024 Plans:\nInvestigate the characterizati...


In [11]:
updated_excel_df.to_excel("updated_R1D_with_descriptions.xlsx", index=False)